In [75]:
import pandas as pds
import numpy as np
import os
from IPython.display import display


In [79]:
leaderboards = pds.read_csv(os.path.join(os.getcwd(),"data\\v1\\leaderboards-data.csv"))
games = pds.read_csv(os.path.join(os.getcwd(),"data\\v1\\games-data.csv"))
users = pds.read_csv(os.path.join(os.getcwd(),"data\\v1\\users-data.csv"))

data_frame = leaderboards.merge(games, left_on="gameID", right_on="ID", how="left")
data_frame = data_frame.merge(users, left_on="players", right_on="ID", how="left")

target = data_frame['emulated']
data_frame = data_frame.drop(columns=['runID'])

In [80]:
display(data_frame)

,gameID,categoryID,levelID,date,primaryTime,place,platform,emulated,players,examiner,...,URL,releaseDate,createdDate,numCategories,numLevels,ID_y,name_y,signupDate,location,numRuns
0,j1n8nj91,8249gled,NaN,2022-12-20,108.000,1,vm9vn63k,True,8d4kdg58,0jm4ppnj,...,computer_space,1971-11-01,2022-10-11T15:53:28Z,3,0,8d4kdg58,Jakaryy,2021-11-08T20:34:20Z,us,36.0
1,j1n8nj91,8249gled,NaN,2022-10-08,114.067,2,vm9vn63k,True,0jm4ppnj,0jm4ppnj,...,computer_space,1971-11-01,2022-10-11T15:53:28Z,3,0,0jm4ppnj,potaterhands,2018-06-05T14:39:39Z,NaN,204.0
2,ldejl7j1,w20m70zk,NaN,2023-06-18,434.000,1,NaN,True,xk1lp76j,j204y17x,...,the_dungeon,1975-01-01,2023-05-24T18:46:03Z,1,0,xk1lp76j,RestiveOrphan,2023-05-20T22:30:30Z,us,22.0
3,ldejl7j1,w20m70zk,NaN,2023-05-24,610.000,2,NaN,True,j204y17x,j204y17x,...,the_dungeon,1975-01-01,2023-05-24T18:46:03Z,1,0,j204y17x,PlayingHistory,2023-05-17T04:56:49Z,us,1.0
4,9d389y91,vdopjgy2,NaN,2022-08-19,25.000,1,v06ddw64,False,8vk4w7vj,8vk4w7vj,...,mattel_auto_race,1976-01-01,2022-08-23T03:09:59Z,1,0,8vk4w7vj,ND606,2021-04-27T03:29:39Z,jp,109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847575,268wwry6,wkpgwqvk,NaN,2023-01-22,1661.626,21,8gej2n93,False,"866ol9p8,0jm94yn8",8wknpmq8,...,roboquest,2023-11-07,2021-06-26T20:35:23Z,2,0,NaN,NaN,NaN,NaN,NaN
1847576,268wwry6,wkpgwqvk,NaN,2021-10-21,1793.943,22,8gej2n93,False,"j0vmykwx,j421715x",8wknpmq8,...,roboquest,2023-11-07,2021-06-26T20:35:23Z,2,0,NaN,NaN,NaN,NaN,NaN
1847577,268wwry6,wkpgwqvk,NaN,2021-06-24,1913.647,23,8gej2n93,False,"8gezg1rj,j2wod3nj",8wknpmq8,...,roboquest,2023-11-07,2021-06-26T20:35:23Z,2,0,NaN,NaN,NaN,NaN,NaN
1847578,268wwry6,wkpgwqvk,NaN,2023-01-21,2126.666,24,8gej2n93,False,"j5qkwg6j,",8wknpmq8,...,roboquest,2023-11-07,2021-06-26T20:35:23Z,2,0,NaN,NaN,NaN,NaN,NaN
